# Performance measurement and comm.send vs comm.Send

When we write parallel code, performance is often a concern.
Measuring performance of parallel code can be challenging:

- where do we measure?
- what do we measure?
- how do we make sense of the numbers we've gathered?


We are going to use the case study of when/why to use mpi4py's `comm.Send` vs `comm.send` and combine it with tools for profiling and plotting.

First, our usual boilerplate to get the cluster going:

In [1]:
import logging
import ipyparallel as ipp

# create a cluster
rc = ipp.Cluster(
    engines="mpi",
    n=2,
    log_level=logging.WARNING,
).start_and_connect_sync(activate=True)

  0%|          | 0/2 [00:00<?, ?engine/s]

In [2]:
%%px
# Find out rank, size
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size

print(f"I am rank {rank} / {size}")

[stdout:0] I am rank 0 / 2


[stdout:1] I am rank 1 / 2


We are going to define some functions that implement ping-pong communication:

- node 0 sends a message to node 1
- node 1 receives it and sends it right back
- node 0 receives the reply

In [3]:
%%px

import time

import numpy as np


def ping_pong_send(arr):
    """ping pong implementation with lowercase 'send' and 'recv'"""
    if rank == 0:
        comm.send(arr, dest=1)
        comm.recv(source=1)
    elif rank == 1:
        comm.recv(source=0)
        comm.send(arr, dest=0)
    
def ping_pong_Send(arr):
    """ping pong implementation with pre-allocated 'Send' and 'Recv'"""
    if rank == 0:
        comm.Send(arr, dest=1)
        comm.Recv(arr, source=1)
    elif rank == 1:
        comm.Recv(arr, source=0)
        comm.Send(arr, dest=0)

def ping_pong_repeat(size, iterations, kind="send", dtype=np.float64):
    """Repeat ping_pong a number of times"""
    if rank == 0:
        arr = np.random.random(size).astype(dtype)
    else:
        arr = np.empty(size, dtype=dtype)
    
    if kind == "send":
        f = ping_pong_send
    elif kind == "Send":
        f = ping_pong_Send
    else:
        raise ValueError(f"{kind=}")
    
    for i in range(iterations):
        f(arr)

`measure_one` takes a given size and arguments, and runs a single measurement.
This samples the call a number of times (as done in `timeit`).
For convenience, a short measurement is run first, to determine how many samples we could take in about one second.

In [4]:
def measure_one(size, target_time=1, **kwargs):
    """Runs a single measurement
    
    Given a size and target measurement time,
    estimate how many iterations are needed to take target_time
    Then run with that many iterations, returning the average time per call.
    """
    comm.Barrier()
    
    tic = time.perf_counter()
    ping_pong_repeat(size, 10, **kwargs)
    toc = time.perf_counter()
    # initial measurement to set iterations to measure
    if rank == 0:
        duration = (toc-tic)
        scale = target_time / duration
        if duration > target_time:
            iterations = 0
        else:
            iterations = int(10 * scale)
        comm.send(iterations, dest=1)
    elif rank == 1:
        iterations = comm.recv(source=0)

    if iterations:
        time.sleep(0.25)
        comm.Barrier()
        tic = time.perf_counter()
        ping_pong_repeat(size, iterations, **kwargs)
        toc = time.perf_counter()
        comm.Barrier()
        per_call = (toc - tic) / iterations
    else:
        # initial measurement exceeded threshold, use it
        per_call = (toc - tic) / 10
    return per_call

Next, we collect data by running several measurements for:

- several sizes from 1k to 1M elements
- both implementations

In [5]:
results = []

from tqdm.notebook import tqdm
for size in tqdm(np.logspace(3, 6, 21).astype(int)):
    for kind in ("send", "Send"):
        per_call = rc[:].apply_sync(measure_one, size, kind=kind)
        per_call = per_call[0]
        calls_per_sec = 1 / per_call
        results.append({"kind": kind, "per_call": per_call, "size": size})

  0%|          | 0/21 [00:00<?, ?it/s]

Now we have a `results` list of samples, which we can collect with pandas and plot with altair.

In [6]:
import pandas as pd
import altair as alt

# results = rc[0]["results"]

df = pd.DataFrame(results)

In [7]:
df

,kind,per_call,size
0,send,0.000019,1000
1,Send,0.000002,1000
2,send,0.000020,1412
3,Send,0.000002,1412
4,send,0.000023,1995
5,Send,0.000003,1995
6,send,0.000028,2818
7,Send,0.000004,2818
8,send,0.000034,3981
9,Send,0.000004,3981


We can group them by n, kind to compare times:

In [8]:
df.groupby(["size", "kind"]).per_call.first()

size     kind
1000     Send    0.000002
         send    0.000019
1412     Send    0.000002
         send    0.000020
1995     Send    0.000003
         send    0.000023
2818     Send    0.000004
         send    0.000028
3981     Send    0.000004
         send    0.000034
5623     Send    0.000008
         send    0.000040
7943     Send    0.000014
         send    0.000047
11220    Send    0.000018
         send    0.000037
15848    Send    0.000011
         send    0.000044
22387    Send    0.000015
         send    0.000050
31622    Send    0.000021
         send    0.000066
44668    Send    0.000026
         send    0.000094
63095    Send    0.000035
         send    0.000115
89125    Send    0.000048
         send    0.000283
125892   Send    0.000068
         send    0.000432
177827   Send    0.000101
         send    0.000469
251188   Send    0.000135
         send    0.001115
354813   Send    0.000196
         send    0.001554
501187   Send    0.000291
         send    0.00217

Or see it more easily, plot the times per call with altair:

In [9]:
alt.Chart(df).mark_line().encode(
    x=alt.X("size", scale=alt.Scale(type="log")),
    y=alt.Y("per_call", scale=alt.Scale(type="log")),
    color="kind",
)

alt.Chart(...)

We can also plot the 'speedup' or the ratio of how much quicker `Send` calls are than `send`:

In [10]:
speedup_df = df.groupby(["size", "kind"]).per_call.first().unstack()
speedup_df["speedup"] = speedup_df["send"] / speedup_df["Send"]
alt.Chart(speedup_df.reset_index()).mark_line().encode(
    x=alt.X("size", scale=alt.Scale(type="log")),
    y="speedup",
)

alt.Chart(...)

So we can see that Send is ~3-9 times faster than `send` for numpy arrays.
This is largely because we avoid additional memory allocations for every recv.

In [11]:
rc.cluster.stop_cluster_sync()